In [21]:
import pandas as pd
import numpy as ny
import matplotlib as mat
import matplotlib.pyplot as plt
import scipy as sci
from nltk.stem import PorterStemmer
from nltk.tokenize import SpaceTokenizer
from nltk.corpus import stopwords
from functools import partial
from gensim import corpora
from gensim.models import TfidfModel
import re
import textmining
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
file_raw=open('F:\\trainingdata.txt','r')
file_data=file_raw.read()
data=file_data.split('\n')
dataframe=pd.DataFrame(data)
dataframe.drop(dataframe.index[0], inplace=True)
dataframe.columns =['data']
dataframe['target']=0

In [4]:
def extracting_target(data):
    count=1
    for doc in data['data']:
        if (count<=dataframe['data'].count() and not doc.isspace() and len(doc)>0):
            dataframe['target'][count]=int(doc[0])
        count=count+1

In [6]:
extracting_target(dataframe)

C:\Users\Ashwin\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#dataframe=dataframe[100:500]
dataframe.count()

data      5486
target    5486
dtype: int64

In [8]:
dataframe['target']=dataframe['target'].astype('category')
dataframe['target'].dtype

category

In [19]:
lower_case=lambda x:x.lower()
remove_numb=lambda x: re.sub('[0-9]+', '', x)
stop=stopwords.words()
remove_stop=lambda s: ' '.join(filter(lambda s: not (s in stopwords.words()),
                                      SpaceTokenizer().tokenize(s)))
remove_specialchar=lambda s: re.sub('[^a-zA-Z0-9 \n\.]','',s)
lemm=lambda s: ' '.join(map(lambda t: WordNetLemmatizer().lemmatize(t), SpaceTokenizer().tokenize(s)))
remove_charater=lambda s:re.sub('\\b[a-zA-Z0-9]{1,3}\\b','',s)
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
remove_punctions=lambda s:s.translate(remove_punctuation_map)
dataframe['data']=dataframe['data'].apply(lower_case)
dataframe['data']=dataframe['data'].apply(remove_numb)
dataframe['data']=dataframe['data'].apply(remove_stop)
dataframe['data']=dataframe['data'].apply(lemm)
dataframe['data']=dataframe['data'].apply(remove_specialchar)
dataframe['data']=dataframe['data'].apply(remove_punctions)
dataframe['data']=dataframe['data'].apply(remove_charater)

In [20]:
dataframe.count()

data      5486
target    5486
dtype: int64

In [34]:
tdm = textmining.TermDocumentMatrix()
def termdocumentmatrix():
    for doc in dataframe['data']:
        tdm.add_doc(doc)

In [35]:
termdocumentmatrix()
tdm.write_csv('matrixtdm.csv')

In [9]:
temp=pd.read_csv('matrixtdm.csv')
temp['target']=dataframe['target']
train ,test=train_test_split(temp, test_size = 0.2)

In [10]:
#colsRes = ['target']
#train_target=ny.asarray(train['target'], dtype="|S6")
#test_target=ny.asarray(test['target'], dtype="|S6")
#train.drop(colsRes, axis = 1)
#test.drop(colsRes, axis = 1)

In [ ]:
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train.ix[:, train.columns != 'target'], train['target'] )

In [89]:
predictions=forest.predict(test.ix[:, test.columns != 'target'])
predictions

array([1, 1, 1, ..., 1, 2, 1], dtype=int64)

In [90]:
df_confusion=pd.crosstab(test['target'], predictions,rownames=['Actual'], 
                       colnames=['Predicted'],margins=True)
df_confusion[1:]
print df_confusion
ny.trace(df_confusion[1:][:8])

Predicted    1    2  3  4  5  6  7  8   All
Actual                                     
0            0    0  0  0  0  0  0  0     0
1          511   66  1  4  1  2  0  2   587
2          253   47  3  4  0  0  2  1   310
3           35    6  3  0  0  0  1  1    46
4           16    4  0  1  0  0  0  0    21
5            7    2  0  0  0  0  0  0     9
6           39    8  0  0  1  0  1  0    49
7           31    2  0  0  0  0  1  0    34
8           25   12  1  0  0  0  1  2    41
All        917  147  8  9  2  2  6  6  1097


565

In [88]:
model = tree.DecisionTreeClassifier(criterion='gini')
model = model.fit( train.ix[:, train.columns != 'target'], train['target'] )

In [91]:
predict=model.predict(test.ix[:, test.columns != 'target'])
df_confusion=pd.crosstab(test['target'], predict,rownames=['Actual'], 
                       colnames=['Predicted'],margins=True)
df_confusion[1:]
print df_confusion
ny.trace(df_confusion[1:][:8])

Predicted    1    2   3   4  5   6   7   8   All
Actual                                          
0            0    0   0   0  0   0   0   0     0
1          349  163  16  14  2  16  11  16   587
2          182   90   8  11  0   2   9   8   310
3           23   16   3   0  0   1   1   2    46
4           11    8   0   1  0   0   0   1    21
5            3    4   1   1  0   0   0   0     9
6           27   15   2   0  0   0   1   4    49
7           18    9   2   0  0   0   2   3    34
8           15   11   8   1  0   2   2   2    41
All        628  316  40  28  2  21  26  36  1097


447

In [116]:
a=temp['target'].value_counts()
a

1    2840
2    1596
6     253
3     251
8     206
7     190
4     108
5      41
0       0
Name: target, dtype: int64

In [22]:
dataframe['data'][2]

u' computer terminal system cpml completes sale computer terminal system  said completed sale share common stock warrant acquire additional   share sedio  lugano switzerland dlrs company said warrant exercisable five year purchase price dlrs share computer terminal said sedio right  additional share increase total holding  computer terminal outstanding common stock certain circumstance involving change control company company said condition occur warrant would exercisable price equal  common stock market price time exceed dlrs share computer terminal said sold technolgy right  matrix impact technology including future improvement woodco  houston  dlrs said would continue exclusive worldwide licensee technology woodco company said move part reorganization plan would help  current operation cost ensure product delivery computer terminal make computer generated label form  ticket printer terminal reuter '

In [38]:
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(train.ix[:, train.columns != 'target'])
df_tfidf=pd.DataFrame(tfidf.toarray().transpose(),columns=train.ix[:, train.columns.values != 'target'])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

TypeError: 'Series' objects are mutable, thus they cannot be hashed